<a href="https://colab.research.google.com/github/henrya2/MitsubaPlayground/blob/main/Scripting_a_renderer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mitsuba

!rm scenes.zip
!wget https://rgl.s3.eu-central-1.amazonaws.com/scenes/tutorials/scenes.zip

# importing the zipfile module
from zipfile import ZipFile

# loading the temp.zip and creating a zip object
with ZipFile("./scenes.zip", 'r') as zObject:

    # Extracting all the members of the zip
    # into a specific location.
    zObject.extractall(
        path="../scenes/")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 35.8 MB/s eta 0:00:00
rm: cannot remove 'scenes.zip': No such file or directory
--2023-10-12 12:33:54--  https://rgl.s3.eu-central-1.amazonaws.com/scenes/tutorials/scenes.zip
Resolving rgl.s3.eu-central-1.amazonaws.com (rgl.s3.eu-central-1.amazonaws.com)... 52.219.168.49, 3.5.135.121, 3.5.134.169, ...
Connecting to rgl.s3.eu-central-1.amazonaws.com (rgl.s3.eu-central-1.amazonaws.com)|52.219.168.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9765095 (9.3M) [application/zip]
Saving to: ‘scenes.zip’

scenes.zip          100%[===================>]   9.31M  7.26MB/s    in 1.3s    

2023-10-12 12:33:56 (7.26 MB/s) - ‘scenes.zip’ saved [9765095/9765095]

